<h1 style="text-align:center;">Алгоритмы составления экстрактивной суммаризации</h1>

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt') 
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

import string
from collections import Counter

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from bs4 import BeautifulSoup

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Максат\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Максат\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Максат\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Максат\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
#!pip install pymorphy3
import pymorphy3

In [5]:
import networkx as nx

## 1. Алгоритм на основе вхождения общих слов

Шаги алгоритма:
1. Разбиение текста на предложения.
2. Каждое предложение разбивается на токены(отдельные слова), слова лемматизируются
3. Считается функция схожести каждой пары предложений(отношение числа общих слов к их суммарной длине) -> матрица схожести
4. Отсеиваем предложения, не имеющие общих слов ни с какими другими.
5. На основе матрицы схожести строим граф: вершины - сами предложения, рёбра - наличие схожести между ними.
6. Считаем значимость каждого предложения и составляем экстрактивную суммаризацию из N наиболее значимых предложений.


In [6]:
class CommonWordsSummarizer():
    '''
    Класс, реализующий алгоритм составления экстрактивной суммаризации на основе вхождения общих слов.
    '''
    def __init__(self, language="english", stop_words=False, make_lemmatization=True):
        """
        Инициализация объекта класса суммаризатора.
        
        Параметры:
            language (str): Язык текстов - 'russian' или 'english'
            stop_words (boolean): Удалять или нет стоп-слова
            make_lemmatization (boolean): Проводить или нет лемматизацию слов
        """
        self.language = language
        if (stop_words):  
            self.stop_words = set(stopwords.words(self.language))
        else:
            self.stop_words = None
        
        self.make_lemmatization = make_lemmatization
        self.lemmatizer_ru = pymorphy3.MorphAnalyzer()
        self.lemmatizer_en = WordNetLemmatizer()
        
        
    def __preprocessor(self, text):
        """
        Предобработка текста: 
        1) удаление ссылок, тегов, номеров
        2) приведение к нижнему регистру
        3) удаление стоп-слов
        4) лемматизация
        
        Возвращает:
            строку, полностью предобработанную. 
        """
        text = BeautifulSoup(text, "html.parser").get_text()
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r'\d+', '', text)
        text = text = re.sub('[^a-zA-Zа-яА-Я0-9.\s]', '', text)
        text = re.sub('\.', ' . ', text)
        
        text = text.lower()
        
        
        if (self.stop_words):
            text = ' '.join([word for word in text.split(' ') if word not in self.stop_words])
        
        if (self.make_lemmatization):
            if (self.language=="english"):
                text = ' '.join([self.lemmatizer_en.lemmatize(word, self.__get_wordnet_pos(word)) for word in text.split()])
            else:
                text = ' '.join([self.lemmatizer_ru.parse(word)[0].normal_form for word in text.split()])
            
        return text
    
    def __get_wordnet_pos(self, word):
        """Функция для преобразования POS-тегов из NLTK в формат, подходящий для WordNetLemmatizer"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {
            'J': wordnet.ADJ,
            'N': wordnet.NOUN,
            'V': wordnet.VERB,
            'R': wordnet.ADV
        }
        return tag_dict.get(tag, wordnet.NOUN)
    
    def __make_similarity_matrix(self, sentences):
        """
        Функция создания матрицы схожести между всеми парами предложений текста.
        Параметры:
            sentences: массив предложений(полностью предобработанных)
        Возвращает:
            матрица схожести размера NxN, N - количество предложений
        """
        similarity_matrix = np.zeros((len(sentences), len(sentences)))
        for i in range(len(sentences)):
            for j in range(len(sentences)):
                if (i != j):
                    similarity_matrix[i][j] = self.__get_similarity(sentences[i], sentences[j])
        return similarity_matrix
    
    
    def __get_similarity(self, sent_1, sent_2):
        """
        Функция, оценивающая схожесть двух предложений на основании количества общих слов.
        Параметры:
            sent_1, sent_2 - две строки(предложения)
        Возвращает: 
            - одно число, схожесть
        """
        set_1 = set(sent_1.split())
        set_2 = set(sent_2.split())
        
        return len(set_1 & set_2) / (len(sent_1.split() + sent_2.split()))
    
    
    def summarize(self, text, N=3):
        """
        Функция, возвращающая экстрактивную суммаризацию текста.
        Параметры:
            text - строка, м.б. непредобработанная.
            N - количество предложений, которые войдут в суммаризацию.
        Возвращает:
            строку, содержащую N предложений - экстрактивная суммаризация
        """
        
        # Предобработка текста
        preprocessed_text = self.__preprocessor(text)
        # Разбиение текста на предложения
        sentences = sent_tokenize(preprocessed_text)
        
        # Удаление точки и пробела в конце каждого предложения
        for i in range(len(sentences)):
            sentences[i] = sentences[i][0:len(sentences[i])-2]
        
        # Нахождение матрицы схожести
        similarity_matrix = self.__make_similarity_matrix(sentences)
        
        # Создаем граф, вершины - предложения, рёбра - схожесть
        graph = nx.from_numpy_array(similarity_matrix)
        # Вычисляем значимость каждого предложения
        scores = nx.pagerank(graph)
        
        scores = list(scores.items())
        # Сортируем предложения по их значимости
        scores.sort(reverse=True, key=lambda x: x[1])
        
        # Отбираем N самых значимых предложений и сортируем их по номерам(порядку в исходном тексте)
        best_N_scores = scores[0:N]
        best_N_scores.sort(key=lambda x: x[0])

        sentences = sent_tokenize(text)
        
        summarization = ' '.join([sentences[best_N_scores[i][0]] for i in range(N)])
        
        return summarization

        
        

<>:39: SyntaxWarning: invalid escape sequence '\s'
<>:40: SyntaxWarning: invalid escape sequence '\.'
<>:39: SyntaxWarning: invalid escape sequence '\s'
<>:40: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Максат\AppData\Local\Temp\ipykernel_7080\2641881868.py:39: SyntaxWarning: invalid escape sequence '\s'
  text = text = re.sub('[^a-zA-Zа-яА-Я0-9.\s]', '', text)
C:\Users\Максат\AppData\Local\Temp\ipykernel_7080\2641881868.py:40: SyntaxWarning: invalid escape sequence '\.'
  text = re.sub('\.', ' . ', text)


In [7]:
model_en = CommonWordsSummarizer(stop_words=True,language='english')
text_en = "Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in the same sport doesn't mean that you have to be friends with everyone just because you're categorized, you're a tennis player, so you're going to get along with tennis players. I think every person has different interests. I have friends that have completely different jobs and interests, and I've met them in very different parts of my life. I think everyone just thinks because we're tennis players we should be the greatest of friends. But ultimately tennis is just a very small part of what we do. There are so many other things that we're interested in, that we do.'"
model_en.summarize(text_en, 5)

"Maria Sharapova has basically no friends as tennis players on the WTA Tour. I have not a lot of friends away from the courts.' Is it different on the men's tour than the women's tour? 'No, not at all. I have friends that have completely different jobs and interests, and I've met them in very different parts of my life."

In [8]:
model_ru = CommonWordsSummarizer(stop_words=True,language='russian', make_lemmatization=True)
text_ru = "ПФЛ выступила с заявлением по поводу ситуации с завершением сезона-2019/20. Лига отмечает, что доиграть сезон, прерванный из-за пандемии короновируса, проблематично. Клубы в текущий и ближайший период не могут проводить тренировочные мероприятия, существует неопределённость по срокам возобновления соревнований, кроме того, у клубов возникнут дополнительные организационные и финансовые издержки в случае продления паузы. Клубы ПФЛ в ходе видеоконференци в апреле отметили, что для проведения оставшихся игр им потребуется период в 1,5 месяца, а также не меньше трех недель на тренировки для набора формы. Совет и администрация ПФЛ призвали объявить сезон завершённым и предоставить право выхода в ФНЛ тем клубам, которые имеют лучшие спортивные показатели и соответствуют необходимым требованиям. Ранее большинство клубов в ФНЛ также проголосовали за досрочное завершение сезона."
model_ru.summarize(text_ru, 2)

'ПФЛ выступила с заявлением по поводу ситуации с завершением сезона-2019/20. Ранее большинство клубов в ФНЛ также проголосовали за досрочное завершение сезона.'

## 2. Алгоритм на основе обученных векторных представлений

In [13]:
#from gensim.models import Word2Vec
from gensim.models import FastText

from sklearn.metrics.pairwise import cosine_similarity

ImportError: cannot import name 'triu' from 'scipy.linalg.lapack' (F:\Downloads\Anaconda\Lib\site-packages\scipy\linalg\lapack.py)

In [10]:
class CosineSimilaritySummarizer():
    def __init__(self, model='Word2Vec', language="english", stop_words=False, make_lemmatization=True):
        """
        Инициализация объекта класса суммаризатора.
        
        Параметры:
            model (str): Используемая модель векторизации слов - 'Word2Vec' или 'FastText'
            language (str): Язык текстов - 'russian' или 'english'
            stop_words (boolean): Удалять или нет стоп-слова
            make_lemmatization (boolean): Проводить или нет лемматизацию слов
        """
        self.model = model
        self.vectorizer = None
        
        self.language = language
        if (stop_words):  
            self.stop_words = set(stopwords.words(self.language))
        else:
            self.stop_words = None
        
        self.make_lemmatization = make_lemmatization
        self.lemmatizer_ru = pymorphy3.MorphAnalyzer()
        self.lemmatizer_en = WordNetLemmatizer()
        
    def __get_wordnet_pos(self, word):
        """Функция для преобразования POS-тегов из NLTK в формат, подходящий для WordNetLemmatizer"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {
            'J': wordnet.ADJ,
            'N': wordnet.NOUN,
            'V': wordnet.VERB,
            'R': wordnet.ADV
        }
        return tag_dict.get(tag, wordnet.NOUN)
    
    def __preprocessor(self, text):
        """
        Предобработка текста: 
        1) удаление ссылок, тегов, номеров
        2) приведение к нижнему регистру
        3) удаление стоп-слов
        4) лемматизация
        
        Возвращает:
            строку, полностью предобработанную. 
        """
        
        text = BeautifulSoup(text, "html.parser").get_text()
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r'\d+', '', text)
        text = text = re.sub('[^a-zA-Zа-яА-Я0-9.\s]', '', text)
        text = re.sub('\.', ' . ', text)
        
        text = text.lower()
        
        
        if (self.stop_words):
            text = ' '.join([word for word in text.split(' ') if word not in self.stop_words])
        
        if (self.make_lemmatization):
            if (self.language=="english"):
                text = ' '.join([self.lemmatizer_en.lemmatize(word, self.__get_wordnet_pos(word)) for word in text.split()])
            else:
                text = ' '.join([self.lemmatizer_ru.parse(word)[0].normal_form for word in text.split()])
            
        return text
    
    
    def fit(self, texts):
        """
        Функция обучения модели векторизации слов 
        Параметры:
            texts (array) - массив текстов(строк, м.б. непредобработанными)
        """
        # предобрабатываем каждый текст
        preprocessed_texts = []
        for text in texts:
            text = self.__preprocessor(text)
            preprocessed_texts.append([word for word in text.split() if word != '.'])
           
        # обучаем на полученных словах модели векторизации
        
        if (self.model == "Word2Vec"):
            self.vectorizer = Word2Vec(sentences=preprocessed_texts, min_count=5, vector_size=100)
        elif (self.model == "FastText"):
            self.vectorizer = FastText(sentences=preprocessed_texts, min_count=5, vector_size=100)
    
    
    def __get_vec(self, word):
        """
        Получения вектора слова. Если слово не знакомо -> возвращается нулевой вектор
        """
        if word in self.vectorizer.wv:
            return self.vectorizer.wv[word]
        else:
            return np.array([0]*100, dtype='float32')
        
        
    def __get_similarity(self, sent_1, sent_2):
        """
        Функция, оценивающая схожесть двух предложений на основании косинусного расстояния
        Параметры:
            sent_1, sent_2 - две строки(предложения)
        Возвращает: 
            - одно число, косинусное расстояние
        """
        # Инициализируем векторные представления предложений
        first_vector = np.array([0]*100, dtype='float32')
        second_vector = np.array([0]*100, dtype='float32')
        
        # Получим списки слов каждого предложения
        words_1 = sent_1.split()
        words_2 = sent_2.split()
        
        # Вектор предложения - сумма векторов его слов, деленная на количество слов
        for word in words_1:
            first_vector += self.__get_vec(word)
        first_vector /= len(words_1)
            
        for word in words_2:
            second_vector += self.__get_vec(word)
        second_vector /= len(words_2)
        
        return cosine_similarity(first_vector.reshape((1, 100)), second_vector.reshape((1, 100)))[0][0]
        
    
    def __make_similarity_matrix(self, sentences):
        similarity_matrix = np.zeros((len(sentences), len(sentences)))
        for i in range(len(sentences)):
            for j in range(len(sentences)):
                if (i != j):
                    similarity_matrix[i][j] = self.__get_similarity(sentences[i], sentences[j])
        return similarity_matrix
    
    
    def summarize(self, text, N=3):
        preprocessed_text = self.__preprocessor(text)
        sentences = sent_tokenize(preprocessed_text)
        
        for i in range(len(sentences)):
            sentences[i] = sentences[i][0:len(sentences[i])-2]
        
        similarity_matrix = self.__make_similarity_matrix(sentences)
        
        graph = nx.from_numpy_array(similarity_matrix)
        scores = nx.pagerank(graph)
        
        scores = list(scores.items())
        scores.sort(reverse=True, key=lambda x: x[1])
        
        best_N_scores = scores[0:N]
        best_N_scores.sort(key=lambda x: x[0])

        sentences = sent_tokenize(text)
        
        summarization = ' '.join([sentences[best_N_scores[i][0]] for i in range(N)])
        
        return summarization

    

In [11]:
df = pd.read_csv('tennis_articles_v4.csv')
df = pd.DataFrame(df['article_text'])
df

,article_text
0,Maria Sharapova has basically no friends as te...
1,"BASEL, Switzerland (AP), Roger Federer advance..."
2,Roger Federer has revealed that organisers of ...
3,Kei Nishikori will try to end his long losing ...
4,"Federer, 37, first broke through on tour over ..."
5,Nadal has not played tennis since he was force...
6,"Tennis giveth, and tennis taketh away. The end..."
7,Federer won the Swiss Indoors last week by bea...


In [12]:
model = CosineSimilaritySummarizer(stop_words=True)

In [13]:
model.fit(df['article_text'].values)

In [14]:
model.summarize(text_en, 3)

"Maria Sharapova has basically no friends as tennis players on the WTA Tour. 'No, not at all. I have friends that have completely different jobs and interests, and I've met them in very different parts of my life."